# Service Ticket Form: Character Capacity Analysis

## Objective
Calculate the character capacity for three text fields in the Service Ticket form:
- **GENERAL DESCRIPTION [TECH TO FILL]**
- **MATERIALS & SERVICES [TECH TO FILL]**
- **LABOR DESCRIPTION [TECH TO FILL]**

The fields should support both web display (modal editors) and physical print forms, with optimal character limits for readability and PDF fit.

In [ ]:
import pandas as pd
import numpy as np

# ============================================================================
# SECTION 1: FORM FACTOR & PHYSICAL DIMENSIONS
# ============================================================================

print("=" * 80)
print("SECTION 1: FORM FACTOR & PHYSICAL DIMENSIONS")
print("=" * 80)

# Standard Letter size (8.5" x 11")
PAPER_WIDTH_INCH = 8.5
PAPER_HEIGHT_INCH = 11.0

# Typical margins (0.75" on all sides for professional forms)
MARGIN_LEFT_INCH = 0.75
MARGIN_RIGHT_INCH = 0.75
MARGIN_TOP_INCH = 0.75
MARGIN_BOTTOM_INCH = 0.75

# Usable area
USABLE_WIDTH_INCH = PAPER_WIDTH_INCH - MARGIN_LEFT_INCH - MARGIN_RIGHT_INCH
USABLE_HEIGHT_INCH = PAPER_HEIGHT_INCH - MARGIN_TOP_INCH - MARGIN_BOTTOM_INCH

# Convert to pixels (96 DPI standard for screen/print)
DPI = 96
USABLE_WIDTH_PX = USABLE_WIDTH_INCH * DPI
USABLE_HEIGHT_PX = USABLE_HEIGHT_INCH * DPI

print(f"\nPaper Size: {PAPER_WIDTH_INCH}\" × {PAPER_HEIGHT_INCH}\"")
print(f"Margins: {MARGIN_LEFT_INCH}\" all sides")
print(f"Usable Area: {USABLE_WIDTH_INCH}\" × {USABLE_HEIGHT_INCH}\" = {USABLE_WIDTH_PX:.0f}px × {USABLE_HEIGHT_PX:.0f}px")

# Font specifications
FONT_FAMILY = "Courier New"  # Monospace for notebook feel
FONT_SIZE_PT = 11
FONT_SIZE_PX = (FONT_SIZE_PT / 72) * DPI

# For monospace fonts, average character width ≈ 0.6 × font size (in pixels)
AVG_CHAR_WIDTH_PX = FONT_SIZE_PX * 0.6

# Line height (typically 1.4-1.5x font size for readability)
LINE_HEIGHT_MULTIPLIER = 1.5
LINE_HEIGHT_PX = FONT_SIZE_PX * LINE_HEIGHT_MULTIPLIER

print(f"\nFont: {FONT_FAMILY}, {FONT_SIZE_PT}pt ({FONT_SIZE_PX:.1f}px)")
print(f"Average Char Width: {AVG_CHAR_WIDTH_PX:.2f}px (monospace)")
print(f"Line Height: {LINE_HEIGHT_PX:.2f}px (1.5x multiplier)")

# Field-specific dimensions
# Assuming three fields stacked vertically, each gets ~1/3 of usable height
FIELD_HEIGHT_PX = USABLE_HEIGHT_PX / 3.5  # Account for headers and spacing

print(f"\nAllocated Height per Field: {FIELD_HEIGHT_PX:.0f}px")


In [ ]:
# ============================================================================
# SECTION 2: COMPUTE CHARACTER CAPACITY PER FIELD
# ============================================================================

print("\n" + "=" * 80)
print("SECTION 2: CHARACTER CAPACITY CALCULATIONS")
print("=" * 80)

# Calculate characters per line
CHARS_PER_LINE = int(USABLE_WIDTH_PX / AVG_CHAR_WIDTH_PX)

# Calculate lines per field
LINES_PER_FIELD = int(FIELD_HEIGHT_PX / LINE_HEIGHT_PX)

# Total characters (conservative: account for word wrap and spacing)
# Word wrap reduces effective characters per line by ~5-10%
WRAP_EFFICIENCY = 0.95
CHARS_PER_LINE_EFFECTIVE = int(CHARS_PER_LINE * WRAP_EFFICIENCY)

# Total capacity
TOTAL_CHARS_PER_FIELD = CHARS_PER_LINE_EFFECTIVE * LINES_PER_FIELD

print(f"\nCalculations:")
print(f"  Usable Width: {USABLE_WIDTH_PX:.0f}px ÷ {AVG_CHAR_WIDTH_PX:.2f}px/char = {CHARS_PER_LINE} chars/line")
print(f"  After 5% wrap overhead: {CHARS_PER_LINE_EFFECTIVE} chars/line effective")
print(f"  Field Height: {FIELD_HEIGHT_PX:.0f}px ÷ {LINE_HEIGHT_PX:.2f}px/line = {LINES_PER_FIELD} lines")
print(f"  Total Capacity: {CHARS_PER_LINE_EFFECTIVE} × {LINES_PER_FIELD} = {TOTAL_CHARS_PER_FIELD} characters")

# Three field specifications
fields = [
    {
        "Name": "General Description",
        "Code": "GENERAL_DESCRIPTION",
        "Purpose": "What work was done, findings, observations",
        "Recommended_Max": TOTAL_CHARS_PER_FIELD,
        "Web_Editor_Max": 800,  # Generous for digital editing
        "Print_Display_Max": TOTAL_CHARS_PER_FIELD,
    },
    {
        "Name": "Materials & Services",
        "Code": "MATERIALS_SERVICES",
        "Purpose": "List of parts used, services rendered, charges",
        "Recommended_Max": TOTAL_CHARS_PER_FIELD,
        "Web_Editor_Max": 800,
        "Print_Display_Max": TOTAL_CHARS_PER_FIELD,
    },
    {
        "Name": "Labor Description",
        "Code": "LABOR_DESCRIPTION",
        "Purpose": "Labor hours, technician work summary, notes",
        "Recommended_Max": TOTAL_CHARS_PER_FIELD,
        "Web_Editor_Max": 800,
        "Print_Display_Max": TOTAL_CHARS_PER_FIELD,
    },
]

df_fields = pd.DataFrame(fields)

print("\n" + "-" * 80)
print("FIELD CAPACITY SUMMARY TABLE")
print("-" * 80)
print(df_fields.to_string(index=False))

# Detailed breakdown table
print("\n" + "-" * 80)
print("DETAILED BREAKDOWN (PRINT FORM)")
print("-" * 80)

breakdown_data = []
for field in fields:
    breakdown_data.append({
        "Field": field["Name"][:20],
        "Purpose": field["Purpose"][:35],
        "Max Chars (Print)": field["Print_Display_Max"],
        "Chars/Line": CHARS_PER_LINE_EFFECTIVE,
        "Lines Available": LINES_PER_FIELD,
        "Paragraph Breaks": max(1, LINES_PER_FIELD // 3)
    })

df_breakdown = pd.DataFrame(breakdown_data)
print(df_breakdown.to_string(index=False))


In [ ]:
# ============================================================================
# SECTION 3: WEB EDITOR VS. PRINT FORM BALANCE
# ============================================================================

print("\n" + "=" * 80)
print("SECTION 3: WEB EDITOR VS. PRINT FORM CAPACITY")
print("=" * 80)

# For web modal editors, we allow more text (scrollable, flexible layout)
# For print forms, we constrain to fit on single page without overflow

WEB_EDITOR_LIMIT = 800  # Current in code
PRINT_FORM_LIMIT = TOTAL_CHARS_PER_FIELD

print(f"\n**WEB MODAL EDITOR**: {WEB_EDITOR_LIMIT} characters")
print(f"  → Allows detailed entry, scrollable dialog")
print(f"  → User sees character counter with remaining count")
print(f"  → Supports multiple paragraphs, line breaks")

print(f"\n**PRINT FORM DISPLAY**: {PRINT_FORM_LIMIT} characters")
print(f"  → Constrained to fit on 1 page (no overflow)")
print(f"  → Monospace font at {FONT_SIZE_PT}pt for readability")
print(f"  → Maximum {LINES_PER_FIELD} lines of text")
print(f"  → {CHARS_PER_LINE_EFFECTIVE} characters per line")

# Strategy
print("\n**STRATEGY**:")
print(f"  1. Store up to {WEB_EDITOR_LIMIT} chars in database (web input)")
print(f"  2. On print, truncate to {PRINT_FORM_LIMIT} chars for display fit")
print(f"  3. Modal dialog shows truncation message if needed")

# Validation thresholds
print(f"\n**VALIDATION THRESHOLDS**:")
print(f"  ✓ Optimal: ≤ {PRINT_FORM_LIMIT // 2} chars (fits comfortably)")
print(f"  ⚠ Caution: {PRINT_FORM_LIMIT // 2} - {PRINT_FORM_LIMIT} chars (fills most space)")
print(f"  ❌ Warning: > {PRINT_FORM_LIMIT} chars (truncation on print)")


## Section 4: Notebook-Style Dialog CSS & Styling

The three text editors should have a **notebook appearance** with:
- Monospace font (Courier New) for professional, ledger-like look
- Ruled lines or grid background (optional)
- Padding/spacing mimicking notebook margins
- Character counter showing remaining capacity
- Visual feedback on nearing limits

Below is the recommended CSS for the modal editors in `pages/tickets.py`.

In [ ]:

# Notebook-style CSS for modal editors
notebook_dialog_css = """
<style>
  /* Notebook-style text editor dialog */
  .gcc-notebook-dialog .q-textarea__control,
  .gcc-notebook-dialog textarea {
    font-family: 'Courier New', monospace;
    font-size: 11pt;
    line-height: 1.5;
    background-color: #f8f8f0;  /* Off-white, paper-like */
    color: #1a1a1a;
    padding: 12px;
    border: 1px solid #ddd;
    border-radius: 2px;
    background-image: 
      linear-gradient(0deg, #e8e8e0 1px, transparent 1px);  /* Ruled lines */
    background-size: 100% 24px;  /* Line spacing matches 1.5x line-height */
    background-position: 0 12px;  /* Align with text top */
  }
  
  .gcc-notebook-dialog textarea {
    resize: vertical;
  }
  
  /* Character counter styling */
  .gcc-char-counter {
    font-size: 0.875rem;
    color: #666;
    text-align: right;
    margin-top: 4px;
  }
  
  .gcc-char-counter.warning {
    color: #ff9800;  /* Orange for caution */
    font-weight: 600;
  }
  
  .gcc-char-counter.danger {
    color: #f44336;  /* Red for near-limit */
    font-weight: 600;
  }
  
  /* Dialog title */
  .gcc-notebook-dialog .text-lg {
    font-family: 'Courier New', monospace;
  }
</style>
"""

print("CSS for Notebook-Style Modal Editors:")
print(notebook_dialog_css)


## Section 5: Implementation Summary & Recommendations

### Character Limits (Updated)**

In [ ]:
# ============================================================================
# SECTION 5: FINAL RECOMMENDATIONS
# ============================================================================

print("\n" + "=" * 80)
print("SECTION 5: IMPLEMENTATION RECOMMENDATIONS")
print("=" * 80)

recommendations = {
    "Web Modal Editor": {
        "Max Characters": 800,
        "Height": "220px (autogrow)",
        "Font": "Courier New, 11pt",
        "Features": ["Character counter", "Live character count", "Maxlength validation"]
    },
    "Print Form Display": {
        "Max Characters": PRINT_FORM_LIMIT,
        "Height": f"~{LINES_PER_FIELD} lines",
        "Font": "Courier New, 11pt, monospace",
        "Features": ["Ruled background (optional)", "Overflow handling (truncate + note)"]
    },
    "Database Storage": {
        "Max Characters": 2000,
        "Rationale": "Allow growth; truncate on print if needed",
        "Validation": "Warn at 800, truncate at 2000 for safety"
    }
}

for context, specs in recommendations.items():
    print(f"\n{context}:")
    for key, value in specs.items():
        if isinstance(value, list):
            print(f"  {key}:")
            for item in value:
                print(f"    • {item}")
        else:
            print(f"  {key}: {value}")

# Summary table
print("\n" + "-" * 80)
print("FINAL CAPACITY MATRIX")
print("-" * 80)

summary_data = {
    "Context": ["Web Editor", "Print Form", "Database"],
    "General Description": [800, PRINT_FORM_LIMIT, 2000],
    "Materials & Services": [800, PRINT_FORM_LIMIT, 2000],
    "Labor Description": [800, PRINT_FORM_LIMIT, 2000],
}

df_summary = pd.DataFrame(summary_data)
print(df_summary.to_string(index=False))

print("\n" + "=" * 80)
print("CONCLUSION")
print("=" * 80)
print(f"""
✓ WEB MODAL EDITORS: Keep current 800-char limit (generous for data entry)
✓ PRINT FORMS: Display up to {PRINT_FORM_LIMIT} characters (fits on 1 page, ~{LINES_PER_FIELD} lines)
✓ PRINT OVERFLOW: If text exceeds {PRINT_FORM_LIMIT} chars, truncate + add "[...See Full Text]" note
✓ NOTEBOOK STYLE: Use monospace, ruled background, character counter
✓ VALIDATION: Show real-time counter; warn at 75% capacity, error at 100%
""")
